<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [24]:
##### Your Code Here #####

# Load Data
import pandas as pd 
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn+(1).csv") 
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [25]:
df.shape

(7043, 21)

In [26]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [5]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.8MB/s eta 0:00:01
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp37-none-any.whl size=73200 sha256=ebbd902e6cbbd9ea498b78666f8141b9b693627c0749c6738f239477e871def2
  Stored in directory: /Users/anitasharma/Library/Caches/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15355 sha256=5a1e66c72e3a2eb0d0d235898de1478f0ae908ae8c9c51ea98d36c39c9233606
  Stored in directory: /Users/anitasharma/Library/Caches/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [27]:
df = df.drop(['customerID'], axis=1)

In [28]:

# Convert Y/N to 0/1

columns = ['Partner', 'Dependents', 'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
           'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']
for col in columns:
    df[col] = [1 if x=='Yes' else 0 for x in df[col]]

df.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,1,0,1,0,No phone service,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,Male,0,0,0,34,1,No,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,Male,0,0,0,2,1,No,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,Male,0,0,0,45,0,No phone service,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,0,0,2,1,No,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [29]:

df['InternetService'] = [0 if x=='No' else 1 for x in df['InternetService']]

In [30]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,1,0,1,0,No phone service,1,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,Male,0,0,0,34,1,No,1,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,Male,0,0,0,2,1,No,1,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,Male,0,0,0,45,0,No phone service,1,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,0,0,2,1,No,1,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [31]:
df.TotalCharges.replace([" "], ["0"], inplace= True)
df.TotalCharges = df.TotalCharges.astype(float)
data = pd.get_dummies(df)
data.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,0,1,0,1,0,1,0,0,...,0,1,0,1,0,0,0,0,1,0
1,0,0,0,34,1,1,1,0,1,0,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,2,1,1,1,1,0,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,45,0,1,1,0,1,1,...,0,1,0,0,1,0,1,0,0,0
4,0,0,0,2,1,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0


In [32]:
target = "Churn"
X = data.drop(target, axis= 1)
y = data.Churn

In [33]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)
X_train.shape

(5634, 27)

In [39]:
# Baseline
y_train.value_counts(normalize = True)

0    0.734469
1    0.265531
Name: Churn, dtype: float64

In [34]:
# Create Model 
 
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics=['accuracy'])
    return model

In [35]:
model = create_model()

In [36]:

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                448       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 593
Trainable params: 593
Non-trainable params: 0
_________________________________________________________________


In [37]:
results = model.fit(X_train, y_train, 
          validation_data = (X_test, y_test),
          epochs = 50,
          batch_size = 10)

Epoch 1/50
564/564 [==============================] - 1s 2ms/step - loss: 1.2542 - accuracy: 0.7114 - val_loss: 0.4302 - val_accuracy: 0.8070
Epoch 2/50
564/564 [==============================] - 1s 2ms/step - loss: 0.7139 - accuracy: 0.7520 - val_loss: 2.1208 - val_accuracy: 0.4833
Epoch 3/50
564/564 [==============================] - 1s 2ms/step - loss: 0.6836 - accuracy: 0.7614 - val_loss: 0.6820 - val_accuracy: 0.7913
Epoch 4/50
564/564 [==============================] - 1s 2ms/step - loss: 0.6871 - accuracy: 0.7632 - val_loss: 0.4111 - val_accuracy: 0.8105
Epoch 5/50
564/564 [==============================] - 1s 1ms/step - loss: 0.5647 - accuracy: 0.7696 - val_loss: 0.4068 - val_accuracy: 0.8155
Epoch 6/50
564/564 [==============================] - 1s 1ms/step - loss: 0.7331 - accuracy: 0.7687 - val_loss: 0.4332 - val_accuracy: 0.8091
Epoch 7/50
564/564 [==============================] - 1s 1ms/step - loss: 0.6223 - accuracy: 0.7682 - val_loss: 0.3995 - val_accuracy: 0.8062
Epoch 

In [38]:
# Batch size and epochs
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dropout


def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7941101670265198 using {'batch_size': 20, 'epochs': 20}
Means: 0.729148268699646, Stdev: 0.07025667308914788 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7941101670265198, Stdev: 0.013142043944444551 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7926877975463867, Stdev: 0.011471811012854289 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7909130334854126, Stdev: 0.007331425690778756 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7781362056732177, Stdev: 0.022716285060181675 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7868312239646912, Stdev: 0.011606191001418936 with: {'batch_size': 100, 'epochs': 20}


In [40]:
# Optimizer

def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'optimizer': ['adam', 'nadam', 'sgd', 'adadelta', 'adagrad']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7799051523208618 using {'optimizer': 'nadam'}
Means: 0.7772493720054626, Stdev: 0.0303601772602544 with: {'optimizer': 'adam'}
Means: 0.7799051523208618, Stdev: 0.012855598701537642 with: {'optimizer': 'nadam'}
Means: 0.737134051322937, Stdev: 0.015096109026019626 with: {'optimizer': 'sgd'}
Means: 0.7360710144042969, Stdev: 0.016562591853140636 with: {'optimizer': 'adadelta'}
Means: 0.6476949691772461, Stdev: 0.09445003992994261 with: {'optimizer': 'adagrad'}


In [41]:

# Learning Rate 

from tensorflow.keras.optimizers import Adam 

def create_model(lr=0.01):
    
    opt = Adam(learning_rate=lr)
    
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = opt, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'lr': [0.001, 0.01, 0.05, 0.1, 0.25]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7573729515075683 using {'lr': 0.01}
Means: 0.7220350980758667, Stdev: 0.08613713608516971 with: {'lr': 0.001}
Means: 0.7573729515075683, Stdev: 0.06754902962704407 with: {'lr': 0.01}
Means: 0.7447649359703064, Stdev: 0.01629349896101116 with: {'lr': 0.05}
Means: 0.734472119808197, Stdev: 0.013571901901362286 with: {'lr': 0.1}
Means: 0.734472119808197, Stdev: 0.013571901901362286 with: {'lr': 0.25}


In [42]:
# Activation Functions 

def create_model(activation='relu'):
    model = Sequential()
    model.add(Dense(16, input_dim=27, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'activation': ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7886015892028808 using {'activation': 'tanh'}
Means: 0.7151366233825683, Stdev: 0.10144038954652668 with: {'activation': 'softmax'}
Means: 0.7721031069755554, Stdev: 0.032851589644762376 with: {'activation': 'softplus'}
Means: 0.7728050947189331, Stdev: 0.011682946506455951 with: {'activation': 'softsign'}
Means: 0.7293358027935029, Stdev: 0.11571311140779038 with: {'activation': 'relu'}
Means: 0.7886015892028808, Stdev: 0.010870286150432215 with: {'activation': 'tanh'}
Means: 0.7758282423019409, Stdev: 0.034483219099542155 with: {'activation': 'sigmoid'}
Means: 0.7774202227592468, Stdev: 0.012726210387451666 with: {'activation': 'hard_sigmoid'}
Means: 0.7060398697853089, Stdev: 0.09966875283456308 with: {'activation': 'linear'}


In [43]:
# Network Weight Initialization 

def create_model(init_mode='uniform'):
    model = Sequential()
    model.add(Dense(16, input_dim=27, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(8, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'init_mode': ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7988991379737854 using {'init_mode': 'normal'}
Means: 0.7823954582214355, Stdev: 0.02705789630192741 with: {'init_mode': 'uniform'}
Means: 0.7321537733078003, Stdev: 0.044299821781485144 with: {'init_mode': 'lecun_uniform'}
Means: 0.7988991379737854, Stdev: 0.007635098184317405 with: {'init_mode': 'normal'}
Means: 0.734472119808197, Stdev: 0.013571901901362286 with: {'init_mode': 'zero'}
Means: 0.7845254898071289, Stdev: 0.01566978555603363 with: {'init_mode': 'glorot_normal'}
Means: 0.7595026612281799, Stdev: 0.0440496717085699 with: {'init_mode': 'glorot_uniform'}
Means: 0.693445873260498, Stdev: 0.11262902866095392 with: {'init_mode': 'he_normal'}
Means: 0.7064411282539368, Stdev: 0.08828249674963476 with: {'init_mode': 'he_uniform'}


In [44]:
# Dropout Regularization 

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm

def create_model(dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(16, input_dim=27, kernel_initializer='normal', activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'weight_constraint': [1, 2, 3, 4, 5],
              'dropout_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7997866034507751 using {'dropout_rate': 0.3, 'weight_constraint': 1}
Means: 0.7976568818092347, Stdev: 0.009000327625883696 with: {'dropout_rate': 0.0, 'weight_constraint': 1}
Means: 0.7942843198776245, Stdev: 0.00940507300282675 with: {'dropout_rate': 0.0, 'weight_constraint': 2}
Means: 0.7928641557693481, Stdev: 0.011354478356368956 with: {'dropout_rate': 0.0, 'weight_constraint': 3}
Means: 0.7923317670822143, Stdev: 0.014683867997295024 with: {'dropout_rate': 0.0, 'weight_constraint': 4}
Means: 0.7948175072669983, Stdev: 0.006323471972854171 with: {'dropout_rate': 0.0, 'weight_constraint': 5}
Means: 0.7962371945381165, Stdev: 0.010140152596950926 with: {'dropout_rate': 0.1, 'weight_constraint': 1}
Means: 0.7880726814270019, Stdev: 0.007956935370866778 with: {'dropout_rate': 0.1, 'weight_constraint': 2}
Means: 0.7953498840332032, Stdev: 0.009395298895530053 with: {'dropout_rate': 0.1, 'weight_constraint': 3}
Means: 0.7939295649528504, Stdev: 0.00695468167402331 with: {'dropou

In [45]:
# Neurons in Hidden Layer

def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=27, kernel_initializer='normal', kernel_constraint=MaxNorm(3), activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'neurons': [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.797301185131073 using {'neurons': 30}
Means: 0.749201500415802, Stdev: 0.024782705489009776 with: {'neurons': 1}
Means: 0.7680205345153809, Stdev: 0.03826444720882093 with: {'neurons': 5}
Means: 0.7740540981292725, Stdev: 0.02793426654580514 with: {'neurons': 10}
Means: 0.7944617867469788, Stdev: 0.006806700230805996 with: {'neurons': 15}
Means: 0.7949948191642762, Stdev: 0.013170662775937505 with: {'neurons': 20}
Means: 0.7820399045944214, Stdev: 0.027553107405140047 with: {'neurons': 25}
Means: 0.797301185131073, Stdev: 0.012137446447468924 with: {'neurons': 30}
Means: 0.7866540908813476, Stdev: 0.025200230567941555 with: {'neurons': 35}
Means: 0.7665969014167786, Stdev: 0.04252141322379644 with: {'neurons': 40}
Means: 0.7800879836082458, Stdev: 0.03116868263746088 with: {'neurons': 45}
Means: 0.7792022347450256, Stdev: 0.03301365772261211 with: {'neurons': 50}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?